### Environment and Device Check  
Before training the model, it is important to verify that TensorFlow is installed correctly and to identify which devices are available for computation.  
This cell prints the TensorFlow version, lists all logical devices detected by TensorFlow, and reports the number of GPUs available.  
This determines whether training will run on CPU or GPU.


In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Available devices:")
for d in tf.config.list_logical_devices():
    print(" ", d)

print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))


### Install kagglehub  
This project uses Tiny-ImageNet downloaded through **kagglehub**, so we need to install the package before accessing the dataset.  
This cell installs kagglehub inside the current environment.


In [ ]:
!pip install kagglehub


### Download Tiny-ImageNet  
We use `kagglehub` to download the Tiny-ImageNet dataset.  
This block retrieves the dataset, prints the local path, and sets the project root directory so later cells can load images correctly.


In [ ]:
import kagglehub
import os

path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")
print("Dataset downloaded to:", path)

root = os.path.join(path, "tiny-imagenet-200")
print("Using:", root)


### Reorganize Validation Set  
Tiny-ImageNet ships the validation images inside one folder with a separate annotations file.  
This cell reads the annotation file, creates class-specific subfolders, and moves images into their correct class directories.  
This ensures Keras generators correctly map labels during training.


In [ ]:
import os
import shutil
import pandas as pd

val_dir = os.path.join(root, "val")
val_images = os.path.join(val_dir, "images")
annotations_file = os.path.join(val_dir, "val_annotations.txt")

df = pd.read_csv(annotations_file, sep="\t", header=None)
df.columns = ["filename", "class", "x1", "y1", "x2", "y2"]

# Create class dirs
for c in df["class"].unique():
    class_dir = os.path.join(val_dir, c)
    os.makedirs(class_dir, exist_ok=True)

# Move images
for _, row in df.iterrows():
    src = os.path.join(val_images, row["filename"])
    dst = os.path.join(val_dir, row["class"], row["filename"])
    if os.path.exists(src):
        shutil.move(src, dst)

print("Validation set reorganized.")


### Build Train and Validation Generators  
This cell creates `ImageDataGenerator` pipelines for the Tiny ImageNet training and validation sets.  
Images are resized to 64x64 and normalized to the range [0, 1].  
Both generators produce one-hot encoded labels for the 200 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 64
BATCH_SIZE = 64

train_gen = ImageDataGenerator(rescale=1.0/255)
val_gen   = ImageDataGenerator(rescale=1.0/255)

train_generator = train_gen.flow_from_directory(
    os.path.join(root, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_gen.flow_from_directory(
    os.path.join(root, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


### Inspect a Sample Batch  
This cell fetches the next batch from the training generator and verifies:
- The image tensor shape  
- The one hot label shape  
- That each label vector sums to 1  
- What unique class indices appear in the batch  


In [ ]:
x, y = next(train_generator)
print("Batch X:", x.shape)
print("Batch Y:", y.shape)
print("Row sums:", y.sum(axis=1)[:10])
print("Unique labels:", sorted(list(set(y.argmax(axis=1)))))


### Define the Custom ResNet-18 Model  
This cell implements a lightweight ResNet-18 suitable for Tiny-ImageNet.  
It includes:  
- A residual block with optional projection  
- Downsampling at each stage  
- Global average pooling  
- Final softmax classification head  

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, stride=1, use_projection=False, name=None):
    shortcut = x

    x = layers.Conv2D(filters, 3, stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if use_projection:
        shortcut = layers.Conv2D(filters, 1, stride, padding="same", use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def build_resnet18(input_shape=(64, 64, 3), num_classes=200):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 3, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = residual_block(x, 64, stride=1, use_projection=False)
    x = residual_block(x, 64, stride=1, use_projection=False)

    x = residual_block(x, 128, stride=2, use_projection=True)
    x = residual_block(x, 128, stride=1, use_projection=False)

    x = residual_block(x, 256, stride=2, use_projection=True)
    x = residual_block(x, 256, stride=1, use_projection=False)

    x = residual_block(x, 512, stride=2, use_projection=True)
    x = residual_block(x, 512, stride=1, use_projection=False)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)


### Compile the ResNet-18 Model  
We compile using Adam with a stable learning rate.  
Loss is categorical cross entropy because Tiny-ImageNet labels are one hot.  
Accuracy is tracked along with planned later metrics (Top 5 accuracy).


In [ ]:
model = build_resnet18()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.TopKCategoricalAccuracy(k=5, name="top5_accuracy")
    ]
)

model.summary()


### Add Training Callbacks  
We include two callbacks to stabilize training:

1. **EarlyStopping**  
   Stops training when validation accuracy stops improving.  
   This prevents overfitting and wasted compute.

2. **ModelCheckpoint**  
   Saves the best performing model during training.  
   We will reload this model later for final evaluation.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint_path = "best_resnet18.h5"

callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=False
    )
]

### Train the ResNet-18 Model  
We now begin full training using the GPU.  
The model will run for up to 20 epochs, but **EarlyStopping** may stop it earlier if validation accuracy plateaus.

Training includes:
- Feedforward and backprop on the training set  
- Validation tracking every epoch  
- Checkpoint saving for the best model  
- Early stopping to avoid overfitting


In [ ]:
EPOCHS = 20  # GPU-friendly

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)


### Plot Training Curves  
To understand learning behavior, we plot both accuracy and loss for training and validation.  
This helps visualize:
- Whether the model is overfitting  
- Whether accuracy is improving across epochs  
- If loss is decreasing or plateauing  


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy"); plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss"); plt.legend()

plt.show()


### Collect Model Predictions for Confusion Matrix  
We gather predictions for the entire validation set by iterating over the validation generator.  
Steps:  
1. Reverse the `class_indices` mapping to convert integer predictions back to class names.  
2. Loop through all validation batches.  
3. Collect ground truth labels (`y_true`) and predicted labels (`y_pred`).  
4. Convert them to numpy arrays for confusion matrix computation.  


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Get class order
class_indices = train_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}

# Predict all validation samples
y_true = []
y_pred = []

val_generator.reset()

for _ in range(len(val_generator)):
    batch_x, batch_y = next(val_generator)
    preds = model.predict(batch_x, verbose=0)
    y_true.extend(np.argmax(batch_y, axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("Prediction collection complete.")


### Confusion Matrix Visualization and Export  
To evaluate how well the model distinguishes between different categories, we compute the confusion matrix across all 200 classes.  
Since plotting all 200 categories at once becomes unreadable, we visualize only the first 20 classes as a heatmap.  

Steps:  
1. Compute the full confusion matrix using all validation predictions.  
2. Extract a smaller 20x20 subset for readable visualization.  
3. Plot a heatmap with seaborn.  
4. Save the full confusion matrix as a CSV file for further analysis or reporting.  


In [ ]:
import pandas as pd

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix shape:", cm.shape)

subset_classes = list(range(20))

cm_subset = cm[np.ix_(subset_classes, subset_classes)]

plt.figure(figsize=(12,10))
sns.heatmap(cm_subset, annot=False, cmap="Blues")
plt.title("Confusion Matrix (first 20 classes)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()



df_cm = pd.DataFrame(cm)
df_cm.to_csv("tiny_imagenet_confusion_matrix.csv", index=False)
print("Confusion matrix saved to tiny_imagenet_confusion_matrix.csv")


### Evaluating the Best Saved Model  
After training with early stopping and checkpointing, the model with the highest validation accuracy is loaded from disk.  
To measure performance, we compute:  

- Final loss on the validation set  
- Top 1 accuracy  
- Top 5 accuracy  

Top 1 accuracy indicates whether the highest softmax probability matches the true class.  
Top 5 accuracy checks whether the correct label appears within the five most likely predictions.  


In [ ]:
best_model = tf.keras.models.load_model("best_resnet18.h5")

results = best_model.evaluate(val_generator)
print("Best model results:", results)
print("Top 1 accuracy:", results[1])
print("Top 5 accuracy:", results[2])
